In [1]:
import numpy as np
import json
from pathlib import Path
from ezc3d import c3d

In [2]:
c3d_folder_path = Path('/Users/wcz/Beanflows/All_Beans/swift4tf/language2motion.gt/data/2017-06-22 c3d/')
out_folder_path = Path(f'/Users/wcz/Beanflows/All_Beans/swift4tf/language2motion.gt/data/motion-viewer/')

In [3]:
def different_label_list(c3d_obj) -> bool:
    labels = c3d_obj['parameters']['POINT']['LABELS']['value']
    num_labels = len(labels)
    diff = not num_labels in [53, 54]
    return diff, num_labels

In [8]:
def c3d2json(c3d_obj) -> str:
    points = c3d_obj.c3d_swig.get_points()

    # find mapping
    labels = c3d_obj['parameters']['POINT']['LABELS']['value']
    bad_labels = [
        "RBAK",
        "LBAK",
        "LAOL",
        "RAOL"
    ]
    bad_label_idxs = [labels.index(label) for label in bad_labels]
    good_labels = [label for label in labels if not label in bad_labels]

    # transpose 2, 53, 3
    points3 = np.transpose(points[:3,:53,0:], (2, 1, 0))

    # identify and remove 4 unconnected points
    good_points_mask = [not x in bad_label_idxs for x in range(points3.shape[1])]
    good_points = points3[:,good_points_mask,:]

    # reshape to 2, 53*3 and divide by 100, and round to 4 decimal places
    frames = (np.reshape(good_points, (good_points.shape[0], good_points.shape[1]*3))/100).round(4)

    # format json
    frames_list = frames[:, :].tolist()

    json_out = {
        'frames': frames_list,
        'interval': 20,
        'markers': good_labels
    }
    json_str = json.dumps(json_out) #, indent=3)
    return json_str

In [5]:
from tqdm import tqdm

In [10]:
different_ones = []
for i in tqdm(range(1, 4000)):
    c3d_path = c3d_folder_path/f'{i:05}_raw.c3d'
    if c3d_path.exists():
        # print(i, c3d_path.name, c3d_path.exists())
        c3d_obj = c3d(str(c3d_path))
        diff, num_labels = different_label_list(c3d_obj)
        if diff:
            print(i, num_labels)
            different_ones.append((i, num_labels))
        else:
            json_str = c3d2json(c3d_obj)
            out_path = out_folder_path/f'{i:05}_raw.json'
            out_path.write_text(json_str)
print(len(different_ones))
print(different_ones)

  0%|          | 16/3999 [00:00<03:39, 18.11it/s]

13 55
14 56
17 56


  2%|▏         | 94/3999 [00:06<03:37, 17.99it/s]

90 56


  3%|▎         | 120/3999 [00:07<03:32, 18.22it/s]

...

130 55


  5%|▌         | 205/3999 [00:13<03:51, 16.36it/s]

203 55


  5%|▌         | 208/3999 [00:13<04:06, 15.35it/s]


KeyboardInterrupt: 

In [ ]:
len(different_ones)